In [156]:
#Import lib
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


In [157]:
#Load Data
tickers = ['GEV', 'TGTX', 'PRVA', 'CEVA', 'GILD', 'KD', 'NRG']
bench_data = ['^GSPC', '^DJI', 'IWM','SPY']

end_date = datetime.now()
start_date = end_date - \
            timedelta(days=1095) # 3 years of data

tickers = yf.download(tickers, start = start_date, end = end_date,auto_adjust=False)
bench_data = yf.download (bench_data, start = start_date, end = end_date,auto_adjust=False)     

print(tickers)
print(bench_data)


[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  4 of 4 completed


Price       Adj Close                                                 \
Ticker           CEVA         GEV        GILD         KD         NRG   
Date                                                                   
2022-03-04  39.410000         NaN   54.723930  11.880000   34.134243   
2022-03-07  37.680000         NaN   53.308502  11.330000   34.543190   
2022-03-08  38.680000         NaN   52.671558  11.290000   34.025185   
2022-03-09  39.139999         NaN   52.193859  11.810000   34.443233   
2022-03-10  39.009998         NaN   51.238441  11.850000   34.216034   
...               ...         ...         ...        ...         ...   
2025-02-24  33.689999  315.910004  111.029999  37.150002  103.430000   
2025-02-25  32.500000  317.690002  111.529999  37.040001  102.440002   
2025-02-26  38.439999  335.239990  110.459999  38.470001  113.330002   
2025-02-27  33.529999  326.279999  111.989998  37.730000  105.110001   
2025-02-28  34.259998  335.179993  114.309998  38.080002  105.70

In [158]:
portfolio_weights = np.ones(len(tickers)) / len(tickers)
portfolio_returns = tickers.pct_change()
portfolio_volatility = portfolio_returns.std() * np.sqrt(252) #annualized volatility

In [159]:
def calc_beta (bench_data, window = 12):
    np_array = bench_data.values
    m = np_array [:,0]
    beta = []
    for ind, col in enumerate(bench_data):
        if ind > 0:
            s = np_array[:, ind]
            covariance = np.cov(s,m)
            beta.append( covariance [0,1] / covariance [1,1])
    return pd.Series(beta, bench_data.columns[1:], name = 'Beta')

bench_data = pd.DataFrame(bench_data)
beta_values = calc_beta(bench_data, window = 12)
print(beta_values)


Price      Ticker
Adj Close  SPY       3.628422e+00
           ^DJI      1.948276e+02
           ^GSPC     3.465388e+01
Close      IWM       9.228049e-01
           SPY       3.454016e+00
           ^DJI      1.948276e+02
           ^GSPC     3.465388e+01
High       IWM       9.209006e-01
           SPY       3.440428e+00
           ^DJI      1.941481e+02
           ^GSPC     3.452984e+01
Low        IWM       9.240951e-01
           SPY       3.458312e+00
           ^DJI      1.950314e+02
           ^GSPC     3.469800e+01
Open       IWM       9.216447e-01
           SPY       3.451256e+00
           ^DJI      1.941617e+02
           ^GSPC     3.459668e+01
Volume     IWM      -6.472376e+04
           SPY      -7.962247e+05
           ^DJI      2.119048e+06
           ^GSPC    -4.681106e+06
Name: Beta, dtype: float64


In [160]:
portfolio_drawdowns = (tickers / tickers.rolling(window = 252).max()) - 1
pportfolio_avg_drawdown = portfolio_drawdowns.mean()
portfolio_max_drawdown = portfolio_drawdowns.min()
print(portfolio_max_drawdown)
print(pportfolio_avg_drawdown)



Price      Ticker
Adj Close  CEVA     -0.528862
           GEV            NaN
           GILD     -0.265898
           KD       -0.325527
           NRG      -0.323089
           PRVA     -0.448154
           TGTX     -0.809143
Close      CEVA     -0.528862
           GEV            NaN
           GILD     -0.276549
           KD       -0.325527
           NRG      -0.342156
           PRVA     -0.448154
           TGTX     -0.809143
High       CEVA     -0.527418
           GEV            NaN
           GILD     -0.272676
           KD       -0.312028
           NRG      -0.346089
           PRVA     -0.449597
           TGTX     -0.808803
Low        CEVA     -0.529581
           GEV            NaN
           GILD     -0.279010
           KD       -0.312235
           NRG      -0.351971
           PRVA     -0.448970
           TGTX     -0.808252
Open       CEVA     -0.537952
           GEV            NaN
           GILD     -0.273906
           KD       -0.311642
           NRG      -0

In [161]:
portfolio_total_returns = (tickers.iloc[-1] / tickers.iloc[0]) - 1
portfolio_annualized_returns = ((1 + portfolio_total_returns) ** (1 /10)) - 1 # Assuming 10 years

In [ ]:
risk_analysis = pd.DataFrame({
    'Ticker': tickers,
    'Portfolio Weight': portfolio_weights,
    'Annualized Returns': portfolio_volatility,
    'Beta against ^GSPC': beta_values['^GSPC'],
    'Beta against ^DJI': beta_values['^DJI'],
    'Beta against IWM': beta_values['IWM'],
    'Average Drawdown': pportfolio_avg_drawdown,
    'Max Drawdown': portfolio_max_drawdown,
    'Total Returns': portfolio_total_returns,
    'Annualized Returns': portfolio_annualized_returns
})

KeyError: '^GSPC'

In [ ]:
Portfolio_Correlation = portfolio_returns.corr(bench_data)
Portfolio_Covariance = portfolio_returns.cov(bench_data)
Portfolio_Tracking_Error = np.sqrt(portfolio_covariance.diagonal())
Portfolio_Sharpe_Ratio = (portfolio_total_returns - 0.02) / portfolio_volatility #Assuming Risk-Free Rate of 2

In [ ]:
bench_volatility = bench_data.pct_change().std() * np.sqrt(252)
volatility_spread = portfolio_volatility - bench_volatility

In [ ]:
Portfolio_risk_table = pd.DataFrame({
    'ETF Ticker': bench_data,
    'Correlation against ETF': Portfolio_Correlation,
    'Covariance against ETF': Portfolio_Covariance,
    'Tracking Error': Portfolio_Tracking_Error,
    'Sharpe Ratio': Portfolio_Sharpe_Ratio,
    'Annualized Volatility Spread': bench_volatility,
})

In [ ]:
correlation_matrix = pd.concat([portfolio_returns, bench_data], axis = 1).corr()


In [ ]:
print("Portfolio Risk Analysis")
print(risk_analysis)

print("\nPortfolio Risk against Benchmarks")
print(Portfolio_risk_table)

print("\nCorrelation Matrix")
print(correlation_matrix)

In [ ]:
#Daily Returns
def calculate_daily_returns(tickers):
    daily_returns = tickers.pct_change()
    return daily_returns

#Cumulative Returns
def cumulative_returns_returns(data):
    return (1+data.pct.change()).cumprod() -1
    

#Max Sharpe Ratio Portfolio
def maximum_sharpe_drawdown(cumulative_returns):
    cov_matrix = returns.cov()
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    ones_vector = np.ones(len(returns.columns))
    weights = inv_cov_matrix.dot(ones_vector) / ones_vector.dot(inv_cov_matrix).do
    return returns.dot(weights)
    
#Risk Parity Portfolio
def risk_parity_portfolio(returns):
    inv_cov_matrix = np.linalg.inv(returns.cov())
    weights = inv_cov_matrix.dot(np.ones(len(returns.columns))) / len(returns.columns)
    return returns.dot(weights)

#Inverse Volatility Weighted Portfolio
def inverse_volatility_weighted_portfolio(returns):
    weights = 1 / returns.std()
    weights /= weights.sum()
    return returns.dot(weights)
